# CA1286

In [ ]:
import pandas as pd
import altair as alt


alt.theme.enable("default")

df = pd.read_csv(
    "..\\CodeAnalysis\\BenchmarkDotNet.Artifacts\\results\\Benchmark.Ca1826-report.csv",
    delimiter=";",
)


def clean_data(df):
    # Replace all instances of " ns" with "" in column: 'Mean'
    df["Mean"] = df["Mean"].str.replace(" ns", "", case=False, regex=False)
    # Replace all instances of "," with "" in column: 'Mean'
    df["Mean"] = df["Mean"].str.replace(",", "", case=False, regex=False)
    # Change column type to float64 for column: 'Mean'
    df = df.astype({"Mean": "float64"})
    # Convert to microsecond
    df["Mean"] = df["Mean"] / 1000
    # # Replace all instances of "Linq" with "" in column: 'Method'
    # df["Method"] = df["Method"].str.replace("Linq", "", case=False, regex=False)
    # # Replace all instances of "Prop" with "" in column: 'Method'
    # df["Method"] = df["Method"].str.replace("Prop", "", case=False, regex=False)
    return df


df_clean = clean_data(df.copy())
df_clean.head()

plot = (
    alt.Chart(df_clean)
    .mark_line(strokeWidth=3)
    .encode(
        y=alt.Y("Mean:Q").title("Execution time, μs"),
        x=alt.X("Length:Q").title("IReadOnlyCollection<T> length").scale(type="log"),
        color="Method:N",
    )
    .properties(
        title={
            "text": "LINQ methods vs direct property access performance",
        },
        height=300,
        width=500,
    )
)

plot.show()
plot.save("results.png", scale_factor=2)

# CA1830

In [ ]:
import pandas as pd
import altair as alt


alt.theme.enable("default")

df = pd.read_csv(
    "..\\CodeAnalysis\\BenchmarkDotNet.Artifacts\\results\\Benchmark.Ca1830-report.csv",
    delimiter=";",
)

def clean_data(df):
    # Clean Mean column
    df["Mean"] = df["Mean"].str.replace(" μs", "", case=False, regex=False)
    df["Mean"] = df["Mean"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Mean": "float64"})
    
    # Clean Allocated column
    df["Allocated"] = df["Allocated"].str.replace(" KB", "", case=False, regex=False)
    df["Allocated"] = df["Allocated"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Allocated": "float64"})
    
    # Replace method names
    df["Method"] = df["Method"].replace({
        "OnlyAppend": "Using strongly-typed Append()",
        "CallToString": "Using ToString()"
    })
    return df

df_clean = clean_data(df.copy())

# Create base charts
base = alt.Chart(df_clean).encode(
    y=alt.Y("Method", sort="x").title(""),
    text=alt.Text(format=".1f")
)

# Time plot
time_plot = base.encode(
    x=alt.X("Mean").title("Execution time, μs")
).properties(
    title={
        "text": "Execution time comparison",
        "subtitle": "1000 Appends"
    },
    height=200,
    width=400
)

# Memory plot
memory_plot = base.encode(
    x=alt.X("Allocated").title("Memory allocated, KiB")
).properties(
    title={
        "text": "Memory allocation comparison",
        "subtitle": "1000 Appends"
    },
    height=200,
    width=400
)

# Add bars and text to both plots
time_plot = time_plot.mark_bar() + time_plot.mark_text(align="left", dx=5)
memory_plot = memory_plot.mark_bar() + memory_plot.mark_text(align="left", dx=5)

# Combine plots horizontally
final_plot = alt.vconcat(time_plot, memory_plot)

final_plot.show()
final_plot.save("results.png", scale_factor=2)

# Hashing

In [ ]:
import pandas as pd
import altair as alt


alt.theme.enable("latimes")

df = pd.read_csv(
    "..\\Hashing\\BenchmarkDotNet.Artifacts\\results\\Benchmark.HashAlgorithms-report.csv",
    delimiter=";",
)


def clean_data(df):
    # Replace all instances of " μs" with "" in column: 'Mean'
    df["Mean"] = df["Mean"].str.replace(" μs", "", case=False, regex=False)
    # Replace all instances of "," with "" in column: 'Mean'
    df["Mean"] = df["Mean"].str.replace(",", "", case=False, regex=False)
    # Change column type to float64 for column: 'Mean'
    df = df.astype({"Mean": "float64"})
    return df


df_clean = clean_data(df.copy())
df_clean.head()

plot = (
    alt.Chart(df_clean)
    .encode(
        y=alt.Y("Method", sort="x").title(""),
        x=alt.X("Mean").title("Execution time, μs"),
        text="Mean",
    )
    .properties(
        title={
            "text": "Hash algorithms comparison",
            "subtitle": "For byte array of size 100 KiB",
        },
        height=500,
        width=800,
    )
)

plot = plot.mark_bar() + plot.mark_text(align="left", dx=5)

plot.show()
plot.save("results.png", scale_factor=2)

# AsNoTracking

In [ ]:
import pandas as pd
import altair as alt


alt.theme.enable("latimes")

df = pd.read_csv(
    "..\\AsNoTracking\\BenchmarkDotNet.Artifacts\\results\\Benchmark.Benchmark-report.csv",
    delimiter=";",
)

def clean_data(df):
    df["Mean"] = df["Mean"].str.replace(" μs", "", case=False, regex=False)
    df["Mean"] = df["Mean"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Mean": "float64"})
    # Clean Ratio column (remove % and convert to float)
    df["Ratio"] = df["Ratio"].astype(str).str.replace("%", "", case=False, regex=False)
    df["Ratio"] = pd.to_numeric(df["Ratio"], errors="coerce")
    return df

df_clean = clean_data(df.copy())

# Main chart: Execution time
main_plot = (
    alt.Chart(df_clean)
    .mark_line()
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Execution time, μs"),
        x=alt.X("WorkersCount").scale(type="log").title("Number of entities in the table"),
        color="Method",
    )
    .properties(
        title="EF Core AsNoTracking() vs Tracking comparison",
        height=300,
        width=800,
    )
)

# Ratio chart
ratio_plot = (
    alt.Chart(df_clean)
    .mark_line(strokeDash=[4,2])
    .encode(
        y=alt.Y("Ratio", title="Ratio (%)"),
        x=alt.X("WorkersCount").scale(type="log").title("Number of entities in the table"),
        color="Method",
    )
    .properties(
        height=150,
        width=800,
    )
)

# Combine vertically
final_plot = alt.vconcat(main_plot, ratio_plot)
final_plot.show()
final_plot.save("results.png", scale_factor=2)

# SqlBulkCopy

In [ ]:
import pandas as pd
import altair as alt


alt.theme.enable("latimes")

df = pd.read_csv(
    "..\\SqlBulkCopy\\BenchmarkDotNet.Artifacts\\results\\Benchmark.Benchmark-report.csv",
    delimiter=";",
)

color_scale = alt.Scale(
    range=["blue", "green", "red"],
)

def clean_data(df):
    df["Mean"] = df["Mean"].str.replace(" ms", "", case=False, regex=False)
    df["Mean"] = df["Mean"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Mean": "float64"})
    
    df["Allocated"] = df["Allocated"].str.replace(" KB", "", case=False, regex=False)
    df["Allocated"] = df["Allocated"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Allocated": "float64"})

    return df

df_clean = clean_data(df.copy())

# Main chart: Execution time
main_plot = (
    alt.Chart(df_clean)
    .mark_line()
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Execution time, ms"),
        x=alt.X("N").title("Array length"),
        color=alt.Color("Method", scale=color_scale),
    )
    .properties(
        title="The fastest way to insert data into SQL Server",
        height=300,
        width=800,
    )
)

# Ratio chart
ratio_plot = (
    alt.Chart(df_clean)
    .mark_line(strokeDash=[4,2])
    .encode(
        y=alt.Y("Allocated", sort="x").title("Allocated memory, KB"),
        x=alt.X("N").title("Array length"),
        color=alt.Color("Method", scale=color_scale),
    )
    .properties(
        height=150,
        width=800,
    )
)

# Combine vertically
final_plot = alt.vconcat(main_plot, ratio_plot)
final_plot.show()
final_plot.save("results.png", scale_factor=2)